<br>
<h1 style = "font-size:25px; font-family:cursive ; font-weight : bold; color : #020296; text-align: center; border-radius: 10px 15px;"> 🚀 Fast Logistic Regression with Intel(R) Extension for Scikit-learn  </h1>
<br>

For classical machine learning algorithms, we often use the most popular Python library, Scikit-learn. We use it to fit models and search for optimal parameters, but scikit-learn sometimes works for hours, if not days. Speeding up this process is something anyone who uses Scikit-learn would be interested in.

I want to show you how to get results faster without changing the code. To do this, we will use another Python library, **[Intel(R) Extension for Scikit-learn](https://github.com/intel/scikit-learn-intelex)**. It accelerates Scikit-learn and does not require you changing the code written for scikit-learn.

I will show you how to speed up your kernel from **14 minutes to 2 minutes** without changes of code!

# 🔨 Installing Intel(R) Extension for Scikit-learn

Let's try to use Intel(R) Extension for Scikit-learn. First, download it. Package also avaialble in conda - please refer to details https://github.com/intel/scikit-learn-intelex

In [ ]:
!pip install scikit-learn-intelex -q --progress-bar off

In [ ]:
import numpy as np
import time
import pandas as pd
import optuna
from sklearn.pipeline import Pipeline
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

# 📋 Reading data and splitting on training and validation datasets

In [ ]:
train = pd.read_csv('../input/tabular-playground-series-jun-2021/train.csv')
test = pd.read_csv('../input/tabular-playground-series-jun-2021/test.csv')
sample_submission = pd.read_csv('../input/tabular-playground-series-jun-2021/sample_submission.csv')

y_train = train['target']
x_train = train.drop(['id','target'], axis=1)
x_test = test.drop(['id'], axis=1)

from sklearn.model_selection import train_test_split
x_train_sub, x_val, y_train_sub, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=42)
print(x_train_sub.shape, x_val.shape)

# 🔍 Defining model and parameters for search optimal model

As Intel(R) Extension for Scikit-learn have pretty fast Logistic Regression now - I can try running optune for with a sufficiently large search to find the optimal model. For prepocessing I use Quantile transformer and for it I will also select the most optimal number of quantiles.

In [ ]:
def logreg_model(params, x_train, y_train, x_test):
    from sklearn.preprocessing import QuantileTransformer
    from sklearn.linear_model import LogisticRegression

    params_quantile = {
        'n_quantiles': params['n_quantiles'],
        'random_state': 46,
    }
    params_logreg = {
        'C': params['C'],
    }
    
    pipe = Pipeline([
        ('quantile', QuantileTransformer(**params_quantile)),
        ('logreg',   LogisticRegression(**params_logreg))
    ])
    pipe.fit(x_train, y_train)
    y_pred = pipe.predict_proba(x_test)
    return y_pred

def objective(trial):
    from sklearn.metrics import log_loss
    
    params = {
        'n_quantiles': trial.suggest_int('n_quantiles', 3, 10),
        'C': trial.suggest_loguniform('C', 1e-5, 1e2),
    }
    
    y_pred = logreg_model(params, x_train_sub, y_train_sub, x_val)
    return log_loss(y_val, y_pred)

# ⚡ Logistic Regression Optuna Optimization with Intel(R) Extension for Scikit-learn

We can take advantage of the performance optimizations of Intel Extension for Scikit-learn by adding just two lines of code before the usual scikit-learn imports:

In [ ]:
from sklearnex import patch_sklearn
patch_sklearn()

Intel(R) Extension for Scikit-learn patching affects performance of specific Scikit-learn functionality. Refer to the [list of supported algorithms and parameters](https://intel.github.io/scikit-learn-intelex/algorithms.html) for details. In cases when unsupported parameters are used, the package fallbacks into original Scikit-learn.

In [ ]:
%%time
t0 = time.time()
study = optuna.create_study(sampler=optuna.samplers.TPESampler(seed=123),
                            direction="minimize",
                            pruner=optuna.pruners.HyperbandPruner())
study.optimize(objective, n_trials=40, show_progress_bar=True)
t1 = time.time()

In [ ]:
print(f"Time for search optimal params: {t1 - t0} sec")
print(f"Best Value: {study.best_trial.value}")
print(f"Best Params: {study.best_params}")

# ⏳ Logistic Regression Optuna Optimization with original Scikit-learn

In order to cancel Intel(R) Extension for Scikit-learn optimizations to run the original Scikit-learn, need to use *unpatch_sklearn*:

In [ ]:
from sklearnex import unpatch_sklearn
unpatch_sklearn()

In [ ]:
%%time
t0 = time.time()
study = optuna.create_study(sampler=optuna.samplers.TPESampler(seed=123),
                            direction="minimize",
                            pruner=optuna.pruners.HyperbandPruner())
study.optimize(objective, n_trials=40, show_progress_bar=True)
t1 = time.time()

In [ ]:
print(f"Time for search optimal params: {t1 - t0} sec")
print(f"Best Value: {study.best_trial.value}")
print(f"Best Params: {study.best_params}")

The search optimal parameters for Logistic Regression model for original Logistic Regression took **almost 14 minutes**. However, the same search using Intel(R) Extension for Scikit-learn took **a little over two minutes**. And most importantly, **the optimal model and quality on the validation dataset are equal**.

# 🎯 Fit final model and submit result

In [ ]:
patch_sklearn()
y_pred = logreg_model(study.best_params, x_train, y_train, x_test)
sample_submission[['Class_1','Class_2', 'Class_3', 'Class_4','Class_5','Class_6', 'Class_7', 'Class_8', 'Class_9']] = y_pred
sample_submission.to_csv(f'logreg.csv', index=False)

# 📜 Conclusions

With Intel(R) Extension for Scikit-learn patching you can:

- Use your scikit-learn code for training and inference without modification;
- Train and predict scikit-learn models and get more time for experiments;
- Get the same quality of predictions.

*Please, upvote if you like.*